## Video Load

In [3]:
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt

from util import *

In [4]:
"""
contours, hierarchy = cv2.findContours(ROI, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                             
contour = contours[0]

epsilon = 0.005 * cv2.arcLength(contour, True)

approx = cv2.approxPolyDP(contour, epsilon, True)

result = cv2.copyTo(frameS04, ROI)

cv2.drawContours(frameS04, [approx], -1, (0,255,0), 1)

            # 필드 위 라인에 대한 처리
            #blurred_s = cv2.GaussianBlur(s, (39, 39), sigmaX=0, sigmaY=0)

            sobel_x = cv2.Sobel(s, cv2.CV_32F, 1, 0)
            sobel_y = cv2.Sobel(s, cv2.CV_32F, 0, 1)

            magnitude = cv2.magnitude(sobel_x, sobel_y).clip(0, 255).astype(np.uint8)

            magnitude = cv2.copyTo(magnitude, field_mask)

            threshold, magnitude = cv2.threshold(magnitude, -1, 255,  cv2.THRESH_BINARY | cv2.THRESH_OTSU) 

            #threshold, s = cv2.threshold(s, -1, 255,  cv2.THRESH_BINARY | cv2.THRESH_OTSU) 

"""

'\ncontours, hierarchy = cv2.findContours(ROI, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)\n                             \ncontour = contours[0]\n\nepsilon = 0.005 * cv2.arcLength(contour, True)\n\napprox = cv2.approxPolyDP(contour, epsilon, True)\n\nresult = cv2.copyTo(frameS04, ROI)\n\ncv2.drawContours(frameS04, [approx], -1, (0,255,0), 1)\n\n            # 필드 위 라인에 대한 처리\n            #blurred_s = cv2.GaussianBlur(s, (39, 39), sigmaX=0, sigmaY=0)\n\n            sobel_x = cv2.Sobel(s, cv2.CV_32F, 1, 0)\n            sobel_y = cv2.Sobel(s, cv2.CV_32F, 0, 1)\n\n            magnitude = cv2.magnitude(sobel_x, sobel_y).clip(0, 255).astype(np.uint8)\n\n            magnitude = cv2.copyTo(magnitude, field_mask)\n\n            threshold, magnitude = cv2.threshold(magnitude, -1, 255,  cv2.THRESH_BINARY | cv2.THRESH_OTSU) \n\n            #threshold, s = cv2.threshold(s, -1, 255,  cv2.THRESH_BINARY | cv2.THRESH_OTSU) \n\n'

In [89]:
def get_ROI_mask(h):
    # H 채널에 대한 히스토그램 구하기
    hist_h = cv2.calcHist([h], [0], None, [256], [0, 256])
    
    # 히스토그램 노이즈 제거
    hist_h = cv2.GaussianBlur(hist_h, (1, 3), sigmaX=0, sigmaY=0)

    # 히스토그램 평균, 분산, 표준 편차
    hist_h /= hist_h.sum()

    mean, variance_l, variance_r = 0, 0, 0
    
    for i in range(len(hist_h)):
        mean += hist_h[i] * i

    for i in range(len(hist_h)):
        variance = (hist_h[i] * (i - mean) ** 2) * 2

        if (i <= mean):
            variance_l += variance
        else:
            variance_r += variance

    SD_l = math.sqrt(variance_l)
    SD_r = math.sqrt(variance_r)

    # 히스토그램을 이미지로 표시
    hist_img = get_histogram_image(hist_h)

    l_weight, r_weight = 1.35, 0.4

    min_range, max_range = 24, 255

    l, r = int(max(mean - SD_l * l_weight, min_range)), int(min(mean + SD_r * r_weight, max_range))

    draw_line_on_histogram_image(hist_img, l, (0, 0, 255))
    draw_line_on_histogram_image(hist_img, r, (0, 0, 255))
    draw_line_on_histogram_image(hist_img, int(mean), (255, 0, 255))

    cv2.imshow("Histogram H", hist_img) 

    # 필드로부터 플레이어 추출
    player_size = 12

    field = cv2.inRange(h, l, r)

    kernel = np.ones((player_size, player_size))
    field = cv2.erode(field, kernel) 

    player_mask = cv2.bitwise_not(field)

    # 필드 영역 추출 
    kernel = np.ones((9, 9))
    field = cv2.dilate(field, kernel)  

    kernel = np.ones((3, 3))
    field = cv2.erode(field, kernel)  

    # 필드 영역 마스크 생성
    field_mask = np.zeros(field.shape, dtype=np.uint8)

    # 외곽선을 찾아서 필드 영역을 단순화
    contours, hierarchy = cv2.findContours(field, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    contour_length = []

    for contour in contours:
        contour_length.append(cv2.arcLength(contour, True))

    if len(contour_length) > 0:
        contour = contours[int(np.argmax(contour_length))]

        cv2.drawContours(field_mask, [cv2.convexHull(contour)], -1, 255, -1) 

    # 필드로부터 플레이어 추출
    player_mask = cv2.copyTo(player_mask, field_mask)            

    return field_mask, player_mask

In [368]:
def load_video(path, ms=25):
    
    # Video Capture 객체 생성
    capture = cv2.VideoCapture(path)

    run, frame = capture.read() # 다음 Frame 읽기
    
    prevS04, prevS16, prevS64 = None, None, None
    
    if run: # Frame을 읽은 경우
        frameS04 = cv2.resize(frame,    (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)
        frameS16 = cv2.resize(frameS04, (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)
        frameS64 = cv2.resize(frameS16, (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)

        prevS04, prevS16, prevS64 = frameS04, frameS16, frameS64

    while capture.isOpened(): # Video Capture가 준비되었는지 확인
        
        run, frame = capture.read() # 다음 Frame 읽기
        
        if run: # Frame을 읽은 경우
            crop = 64

            frame = frame[crop:-crop,:]

            # 영상 사이즈 기준 1920 x 1080

            # 영상의 사이즈 조정 (x1/16 또는 x1/32)
            frameS04 = cv2.resize(frame,    (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)
            frameS16 = cv2.resize(frameS04, (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)
            frameS64 = cv2.resize(frameS16, (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)
                            
            hsv = cv2.cvtColor(frameS04, cv2.COLOR_BGR2HSV)

            h, s, v = cv2.split(hsv)

            field_mask, player_mask = get_ROI_mask(h) 

            # 원근 보정을 위해 필드에서 특징 추출 
            perspective = cv2.equalizeHist(cv2.copyTo(s, field_mask))
            
            height, width = perspective.shape
            
            scaler = (width - height) // 2

            # 정사각형으로 이미지 크롭
            perspective = perspective[:,scaler:-scaler]

            perspective = cv2.resize(perspective, (0, 0), fx=1/16, fy=1/16, interpolation= cv2.INTER_AREA)
            
            height, width = perspective.shape 

            kernel = np.array([[-1] * 3 + [0] * 1 + [+1] * 3] * height) / (3 * height)
            
                # 어파인 행렬 생성
            aff = np.array([[1, 0.5, 0], [0, 1, 0]], dtype=np.float32)
            kernel = cv2.warpAffine(kernel, aff, (9 + int(height * 0.5), height))

            cv2.imshow("kernel", kernel * 255 * 12 * 24)
            #perspective = (np.abs(cv2.filter2D(perspective, cv2.CV_32F, kernel)) > 64).astype(np.uint8) * 255 
            perspective = np.abs(cv2.filter2D(perspective, cv2.CV_32F, kernel)).clip(0, 255).astype(np.uint8)
            
            cv2.imshow("perspective", perspective)
            frameS04 = cv2.copyTo(frameS04, field_mask)
            cv2.imshow("Frame", frameS04) 
            cv2.imshow("mask", cv2.hconcat((field_mask, player_mask)))

            cv2.waitKey(ms) # Millisecond 단위로 대기

            prevS04, prevS16, prevS64 = frameS04, frameS16, frameS64
        else: # 재생이 완료되어 더 이상 Frame을 읽을 수 없는 경우
            break

    capture.release() # Capture 자원 반납
    cv2.destroyAllWindows() # 창 제거

In [369]:
# Video가 저장된 경로 입력
PATH = r"video/soccer2.mp4"

# Video 재생 및 반환 (Numpy Array)
load_video(PATH, ms=20)